In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
from time import time
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('data/train.csv').sort_values('date')
df['recipient_id'] = df['recipient_id'].apply(literal_eval)
df_test = pd.read_csv('data/test.csv').sort_values('date')

In [5]:
df.head()

,sender,sender_id,mid,date,body,recipient_id,recipients
5304,enron_update@concureworkplace.com,124,47361,0001-08-26 22:16:36,The following reports have been waiting for yo...,[377],kimberly.watson@enron.com
5305,enron_update@concureworkplace.com,124,47362,0001-08-27 22:21:02,The following reports have been waiting for yo...,[377],kimberly.watson@enron.com
5306,enron_update@concureworkplace.com,124,47363,0001-08-28 22:25:35,The following reports have been waiting for yo...,[377],kimberly.watson@enron.com
5285,enron_update@concureworkplace.com,124,45909,0001-09-13 22:24:08,Employee Name: Kimberly WatsonReport Name: E...,[377],kimberly.watson@enron.com
8934,enron_update@concureworkplace.com,124,82030,0001-09-17 09:24:00,The following expense report is ready for appr...,[121],barry.tycholiz@enron.com


In [23]:
df.values[10000]

array(['sara.shackleton@enron.com', 83, 180892, '2000-10-03 01:14:00',
       'fax:  713-646-3490\t"Susan Hopkinson" <susan.hopkinson@lovells.com>\t10/03/2000 07:53 AM\t\t \t\t To: <Sara.Shackleton@enron.com>\t\t cc: \t\t Subject: Re: ISDA Master Agreement between Enron North America Corp. andSociieteIndustrielle de Transports Automobile S.A.CONFIDENTIALITY This e-mail and any attachments are confidentialand may also be privileged. If you are not the named recipient,please notify the sender immediately and do not disclose thecontents to another person, use it for any purpose, or storeor copy the information in any medium.----------------------------------------------------------------Please would you send me your fax number.Many thanksSusan----------------------------------------------------------------Lovells (the merged firm of Lovell White Durrant andBoesebeck Droste, practising as Lovells Boesebeck Droste insome jurisdictions) is an international law firm.In the event of any techni

In [7]:
df_test.values[-1]

array([2058, 'chris.germany@enron.com', 22, 347212, '2002-06-24 13:15:28',
       '??-----Original Message-----From: Ivy Kao [mailto:ivy_kao@iroquois.com]Sent: Monday, June 24, 2002 3:14 PMTo: Germany, ChrisCc: robin_zaleski@iroquois.comSubject: Capacity releaseHello Chris:I am just curious to find out if you are planning to release the 35,465dth back to Boston Gas this month.  Please let me know.  Thanks.Ivy'], dtype=object)

In [4]:
from collections import defaultdict

senders_recipients = defaultdict(lambda: set())
for l in df.values:
    for r in l[5]:
        senders_recipients[l[1]].add(r)
        
c = 0
for s, r in senders_recipients.iteritems():
    c += len(r)
print float(c)/125

186.328


In [5]:
from collections import defaultdict

senders_counts = np.zeros(125)
for l in df.values:
    senders_counts[l[1]] += 1

print np.sum(senders_counts**2)

48209709.0


It looks possible (but a bit long) to train one svm per sender.

### Here I implement this idea for each sender, and add the time as 4 additional features.
http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

We aim at builing a ranking generator for each sender. For this we use word2vec to get reasonable features, and we interpret the task as a regression task, using one hot encoding on the recipients. Next we might try to train an embedding on the recipients, which seems to be best solution to fit our needs.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix

def ohe(y):
    seen = set()
    indexes = []
    for l in y:
        for r in l:
            if not r in seen:
                seen.add(r)
                indexes.append(r)
    rindexes = {indexes[i]: i for i in range(len(indexes))}
    
    y_ohe = np.zeros((len(y), len(indexes)), dtype=float)
    for i in range(len(y)):
        l = y[i]
        for r in l:
            y_ohe[i][rindexes[r]] = 1.
    return y_ohe, indexes

def make_dataset(df, w2v, d, split_ratio=.1, min_df=10):
    stemmer = PorterStemmer
    X, X_dates, y = df[['sender_id', 'body']].values, df['date'].values, df['recipient_id'].values
    
    print "stemming..."
    stemmer = PorterStemmer()
    for i in range(len(X)):
        X[i][1] = ' '.join(map(lambda x: str(stemmer.stem(x)), word_tokenize(X[i][1])))
    
    print "vectorizing..."
    #vect = CountVectorizer(min_df=min_df, stop_words='english')
    vect = TfidfVectorizer(min_df=min_df, stop_words='english')
    X_body = vect.fit_transform(X[:,1])
    
    print "w2v..."
    voc = vect.vocabulary_
    coef = np.array([s if s != 0. else 1. for s in X_body.sum(axis=1)])
    translate_matrix = np.empty((len(voc), d), dtype=float)
    for w, i in voc.iteritems():
        translate_matrix[i] = w2v[w] if w in w2v else np.zeros(d)
    vectorized = X_body.dot(translate_matrix)/coef.reshape(-1, 1)
    
    #date_features = np.array([np.array([s[:4], s[5:7], s[8:10], s[11:13]]) for s in X_dates]).astype(float)
    
    X_full = vectorized
    
    X_lol, y_lol = defaultdict(lambda: []), defaultdict(lambda: [])
    recipient_indexes = []
    for i in range(len(X)):
        X_lol[X[i][0]].append(X_full[i])
        y_lol[X[i][0]].append(y[i])
    
    print "one hot encoding..."
    recipient_indexes = []
    for i in range(125):
        y_lol[i], indexes = ohe(y_lol[i])
        recipient_indexes.append(indexes)
        
        
    X_train, X_test, y_train, y_test = [], [], [], []
    
    for i in range(125):
        cur_X = StandardScaler().fit_transform(np.array(X_lol[i]))
        train_size = len(cur_X) - int(.1*len(cur_X))
        X_train.append(cur_X[:train_size])
        X_test.append(cur_X[train_size:])
        cur_y = np.array(y_lol[i])
        y_train.append(cur_y[:train_size])
        y_test.append(cur_y[train_size:])
        if(y_train[i].shape[1] == 1):
            y_train[i] = y_train[i].ravel()
            y_test[i] = y_test[i].ravel()
    
    train_size = len(X) - int(.1*len(X))
    return X_train, X_test, y_train, y_test, recipient_indexes

In [7]:
with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [8]:
from time import time
start = time()
X_train, X_test, y_train, y_test, recipient_indexes = make_dataset(df, w2v, 50)
print time() - start

stemming...
vectorizing...
w2v...
one hot encoding...


/home/daemonginger/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


139.912637949


In [11]:
from sklearn.ensemble import RandomForestRegressor

In [12]:
def ap(recommanded, real):
    real_set = set(real)
    cur = 0.
    n = len(recommanded)
    ans = 0.
    for k in range(1, n+1):
        if recommanded[k-1] in real_set:
            cur += 1
            ans += cur/k
    return ans/min(n, len(real))

In [13]:
from time import time

start = time()
models = []
cur = 0
for i in range(125):
    print("{} done".format(cur))
    print("Next shape is {}".format(y_train[i].shape))
    models.append(RandomForestRegressor(n_estimators=50, max_leaf_nodes=300).fit(X_train[i], y_train[i]))
    cur += 1
print time() - start

y_pred = []
for i in range(125):
    y_pred.append(models[i].predict(X_test[i]))
    
score, c = 0, 0
for i in range(125):
    for j in range(len(y_test[i])):
        real = np.argsort(y_test[i][j])[-10:]
        recommanded = list(np.argsort(y_pred[i][j])[-10:])
        while len(recommanded) < 10:
            recommanded.append(np.max(recommanded) + 1)
        score += ap(recommanded, real)
        c += 1

print("Score on test: {}".format(score/c))

y_pred_train = []
for i in range(125):
    y_pred_train.append(models[i].predict(X_train[i]))
    
score, c = 0, 0
for i in range(125):
    for j in range(len(y_train[i])):
        real = np.argsort(y_train[i][j])[-10:]
        recommanded = list(np.argsort(y_pred_train[i][j])[-10:])
        while len(recommanded) < 10:
            recommanded.append(np.max(recommanded) + 1)
        score += ap(recommanded, real)
        c += 1
        
print("Score on train: {}".format(score/c))

0 done
Next shape is (141, 544)
1 done
Next shape is (351, 123)
2 done
Next shape is (79, 152)
3 done
Next shape is (112, 57)
4 done
Next shape is (105, 351)
5 done
Next shape is (75, 63)
6 done
Next shape is (468, 394)
7 done
Next shape is (99, 10)
8 done
Next shape is (311, 841)
9 done
Next shape is (76, 11)
10 done
Next shape is (126, 67)
11 done
Next shape is (63, 56)
12 done
Next shape is (88, 183)
13 done
Next shape is (365, 143)
14 done
Next shape is (270, 399)
15 done
Next shape is (94, 134)
16 done
Next shape is (255, 111)
17 done
Next shape is (546, 200)
18 done
Next shape is (315, 219)
19 done
Next shape is (91, 211)
20 done
Next shape is (153, 354)
21 done
Next shape is (471, 266)
22 done
Next shape is (2226, 350)
23 done
Next shape is (1313, 199)
24 done
Next shape is (470, 406)
25 done
Next shape is (681, 273)
26 done
Next shape is (151, 83)
27 done
Next shape is (99, 147)
28 done
Next shape is (400, 151)
29 done
Next shape is (474, 112)
30 done
Next shape is (436, 332)
3